# Projet de Fouille de données

## Etape II : Nettoyage des données

Ce script fait suite directement au script d'analyse exploratoire des données (Etape I).

Il est nécessaire de lire ce script précédent, pour comprendre ce qui est réalisé ici.

Dans ce script, nous nettoyons les données, afin de faire disparaitre les problèmes des données manquantes et incohérentes.

### Import des librairies

On commence par importer les librairies.

In [1]:
import pandas
import numpy

### Import des données

Puis, on importe les données de notre jeu de données, et on les "wrap" dans un DataFrame de Pandas.

In [2]:
data = pandas.read_csv("../data/corrupted_data.csv")
dataFrame = pandas.DataFrame(data)

### Nettoyage des données

Nous réalisons différentes formes de transformations pour nettoyer les données.

#### Renommage de la première colonne en "id"

On renomme la première colonne avec un nom correct. 

In [3]:
cleanedDataFrame = dataFrame.rename(columns={'Unnamed: 0': 'id'})

In [4]:
list(cleanedDataFrame)

['id', 'date', 'county', 'state', 'fips', 'cases', 'deaths']

#### Suppression des lignes avec des valeurs manquantes (cellules vides, et "Unknown")

On supprime toutes les lignes dans le jeu de données, qui comportent des données manquantes (cellules vides, et "Unknown") dans au moins une colonne.

In [5]:
cleanedDataFrame = cleanedDataFrame.dropna()

In [6]:
len(cleanedDataFrame)

775346

In [7]:
cleanedDataFrame.count()

id        775346
date      775346
county    775346
state     775346
fips      775346
cases     775346
deaths    775346
dtype: int64

In [8]:
cleanedDataFrame.count() - len(cleanedDataFrame)

id        0
date      0
county    0
state     0
fips      0
cases     0
deaths    0
dtype: int64

In [9]:
cleanedDataFrame = cleanedDataFrame.drop(cleanedDataFrame[cleanedDataFrame["county"]=="Unknown"].index)

In [10]:
len(cleanedDataFrame)

775346

In [11]:
cleanedDataFrame[cleanedDataFrame["county"]=="Unknown"]

,id,date,county,state,fips,cases,deaths


In [12]:
len(cleanedDataFrame[cleanedDataFrame["county"]=="Unknown"]["county"])

0

#### Conversion au même format pour les dates (YYYY.MM.DD => YYYY-MM-DD)

On fait en sorte de passer au même format toutes les dates.

In [13]:
 cleanedDataFrame['date'] = cleanedDataFrame['date'].replace(to_replace = "^([0-9]{4})\.([0-9]{2})\.([0-9]{2})$", value = "\\1-\\2-\\3", regex=True)

In [14]:
cleanedDataFrame[cleanedDataFrame['date'].str.contains('^[0-9]{4}\.[0-9]{2}\.[0-9]{2}$', regex = True, na = False)]

,id,date,county,state,fips,cases,deaths


In [15]:
len(cleanedDataFrame[cleanedDataFrame['date'].str.contains('^[0-9]{4}\.[0-9]{2}\.[0-9]{2}$', regex = True, na = False)]['date'])

0

#### Suppression des "-" dans les colonnes quantitatives (fips, cases, deaths)

On supprime tous les "-" dans les colonnes quantitatives, car aucune de nos données quantitatives ne devraient compoter de colonnes quantitatives.

In [16]:
cleanedDataFrame['fips'] = cleanedDataFrame['fips'].apply(str).replace(to_replace = "^-(.+)$", value = "\\1", regex=True)

In [17]:
cleanedDataFrame[cleanedDataFrame['fips'].apply(str).str.contains('^-.+$', regex = True, na = False)]['fips']

Series([], Name: fips, dtype: object)

In [18]:
len(cleanedDataFrame[cleanedDataFrame['fips'].apply(str).str.contains('^-.+$', regex = True, na = False)]['fips'])

0

In [19]:
cleanedDataFrame['cases'] = cleanedDataFrame['cases'].apply(str).replace(to_replace = "^-(.+)$", value = "\\1", regex=True)

In [20]:
cleanedDataFrame[cleanedDataFrame['cases'].apply(str).str.contains('^-.+$', regex = True, na = False)]['cases']

Series([], Name: cases, dtype: object)

In [21]:
len(cleanedDataFrame[cleanedDataFrame['cases'].apply(str).str.contains('^-.+$', regex = True, na = False)]['cases'])

0

In [22]:
cleanedDataFrame['deaths'] = cleanedDataFrame['deaths'].apply(str).replace(to_replace = "^-(.+)$", value = "\\1", regex=True)

In [23]:
cleanedDataFrame[cleanedDataFrame['deaths'].apply(str).str.contains('^-.+$', regex = True, na = False)]['deaths']

Series([], Name: deaths, dtype: object)

In [24]:
len(cleanedDataFrame[cleanedDataFrame['deaths'].apply(str).str.contains('^-.+$', regex = True, na = False)]['deaths'])

0

#### Transformation des colonnes quantitatives en entiers (fips, cases, deaths)

On passe les types de toutes les colonnes quantitatives de décimal à entier, car aucune de nos données quantitatives ne devraient être de type décimal.

In [25]:
cleanedDataFrame["fips"] = pandas.to_numeric(cleanedDataFrame["fips"], downcast='integer')

In [26]:
cleanedDataFrame["fips"]

0         53061
1         53061
2         53061
3         17031
4         53061
          ...  
800432    56037
800433    56039
800434    56041
800435    56043
800436    56045
Name: fips, Length: 775346, dtype: int32

In [27]:
cleanedDataFrame["cases"] = pandas.to_numeric(cleanedDataFrame["cases"], downcast='integer')

In [28]:
cleanedDataFrame["cases"]

0            1
1            1
2            1
3            1
4            1
          ... 
800432    2098
800433    1739
800434    1187
800435     519
800436     419
Name: cases, Length: 775346, dtype: int32

In [29]:
cleanedDataFrame["deaths"] = pandas.to_numeric(cleanedDataFrame["deaths"], downcast='integer')

In [30]:
cleanedDataFrame["deaths"]

0          0
1          0
2          0
3          0
4          0
          ..
800432    10
800433     2
800434     5
800435     8
800436     2
Name: deaths, Length: 775346, dtype: int16

### Export des données nettoyées

On exporte les données nettoyées dans le répertoire des données du projet.

In [31]:
cleanedDataFrame.to_csv("..\data\cleaned_data.csv", index = False)